**First we import the libraries we will use and then we will import the two dataset: trips and details**

In [ ]:
import pandas as pd
import datetime
import re
import zipfile
import io

**Metadata**

In [ ]:
!wget https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv

--2024-04-29 19:29:33--  https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/metaData.csv
Resolving www.research-collection.ethz.ch (www.research-collection.ethz.ch)... 129.132.8.151
Connecting to www.research-collection.ethz.ch (www.research-collection.ethz.ch)|129.132.8.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272886 (266K) [text/csv]
Saving to: ‘metaData.csv’

metaData.csv        100%[===================>] 266.49K   572KB/s    in 0.5s    

2024-04-29 19:29:34 (572 KB/s) - ‘metaData.csv’ saved [272886/272886]



In [ ]:
trips = (pd.read_csv('/content/metaData.csv',
                     usecols=[0, 1, 2, 6, 7, 8, 9, 13],
                     dtype={
                         'name': 'string',
                         'busNumber': 'Int16',
                         'startTime_iso': 'string',
                         'drivenDistance': 'float32',
                         'busRoute': 'string',
                         'energyConsumption': 'float32',
                         'itcs_numberOfPassengers_mean': 'float32',
                         'temperature_ambient_mean': 'float32',
                         },
                     na_values={'busRoute': '-'})
         .convert_dtypes())

**Details**

In [ ]:
!wget https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip
!unzip /content/ZTBus_compressed.zip

--2024-04-29 19:37:07--  https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/626723/ZTBus_compressed.zip
Resolving www.research-collection.ethz.ch (www.research-collection.ethz.ch)... 129.132.8.151
Connecting to www.research-collection.ethz.ch (www.research-collection.ethz.ch)|129.132.8.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2006877986 (1.9G) [application/zip]
Saving to: ‘ZTBus_compressed.zip’

ZTBus_compressed.zi 100%[===================>]   1.87G  20.6MB/s    in 86s     

2024-04-29 19:38:34 (22.2 MB/s) - ‘ZTBus_compressed.zip’ saved [2006877986/2006877986]

Archive:  /content/ZTBus_compressed.zip
  inflating: B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv  
  inflating: B183_2019-04-30_13-22-07_2019-04-30_17-54-02.csv  
  inflating: B183_2019-05-01_05-58-51_2019-05-01_22-32-30.csv  
  inflating: B183_2019-05-03_02-50-21_2019-05-03_05-53-20.csv  
  inflating: B183_2019-05-03_15-41-57_2019-05-03_23-06-24.csv  
  inflating: B18

In [ ]:
dtype_dict = {
    'gnss_altitude': 'float32',
    'itcs_busRoute': 'string',
    'itcs_numberOfPassengers': 'float32',
    'itcs_stopName': 'string',
    'odometry_vehicleSpeed': 'float32',
    'status_haltBrakeIsActive': 'boolean',
    'status_parkBrakeIsActive': 'boolean',
    'temperature_ambient': 'float32'
}
details_dict = {}
for n in trips.name:
    df_temp = pd.read_csv(n + '.csv',
                          usecols=[0, 3, 7, 8, 9, 12, 21, 22, 23],
                          dtype=dtype_dict,
                          na_values={'itcs_busRoute': '-', 'itcs_stopName': '-'},
                          parse_dates=['time_iso'])
    df_temp['name'] = n
    df_temp['name_short'] = n[:4]
    details_dict[n] = df_temp
details = pd.concat(details_dict.values(), ignore_index=True)

**0. Transformation of some variables**


Transformation for trips

In [ ]:
trips['energyConsumptionKJ']= trips['energyConsumption']/1000
trips['celsius.mean'] = round(trips['temperature_ambient_mean'] - 273.15, 3)

**1. Extract all trips with busRoute 83**


In [ ]:
trips[trips.busRoute == '83']

,name,busNumber,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,temperature_ambient_mean,energyConsumptionKJ,celsius.mean
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,225047.90625,83,1544278016.0,23.475309,280.545,1544278.0,7.395
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,224512.296875,83,1631815936.0,17.415779,279.885,1631815.875,6.735
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,77824.359375,83,540601280.0,23.18182,281.0489,540601.25,7.899
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,225095.796875,83,1692171008.0,20.9641,279.8363,1692171.0,6.686
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,224181.203125,83,1145859968.0,17.212351,287.3445,1145860.0,14.195
...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,223165.0,83,1560888064.0,27.890659,280.6948,1560888.125,7.545
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,190196.0,83,1418846976.0,26.03927,279.7655,1418847.0,6.615
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,224473.40625,83,1611149952.0,24.803841,279.7888,1611150.0,6.639
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,210041.59375,83,1536696960.0,28.785391,279.5283,1536697.0,6.378


**2. Extract all trips where busRoute is not a number**

In [ ]:
trips['busRoute'].unique()

<StringArray>
[<NA>, '31', '33', '72', '46', '32', '83', 'N4', 'N2', 'N1']
Length: 10, dtype: string

In [ ]:
trips[trips['busRoute'].isin(['N4', 'N2', 'N1'])]

,name,busNumber,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,temperature_ambient_mean,energyConsumptionKJ,celsius.mean
533,B183_2021-12-18_23-37-00_2021-12-19_03-38-35,183,76216.0625,N4,481350304.0,9.198582,276.8632,481350.3125,3.713
553,B183_2022-01-07_23-40-43_2022-01-08_03-31-21,183,68557.0625,N2,453625088.0,4.626984,276.9673,453625.09375,3.817
554,B183_2022-01-08_23-40-17_2022-01-09_03-35-32,183,67962.921875,N2,475383296.0,7.495495,278.5645,475383.28125,5.415
561,B183_2022-01-15_23-41-46_2022-01-16_03-40-23,183,77156.703125,N1,525168288.0,6.5125,274.9937,525168.3125,1.844
568,B183_2022-01-21_23-35-40_2022-01-22_03-26-24,183,71917.75,N2,455476000.0,5.357143,275.3073,455476.0,2.157
...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,78567.15625,N1,434776608.0,16.33333,289.255,434776.59375,16.105
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,73427.96875,N2,399773696.0,17.710529,287.3486,399773.6875,14.199
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,72911.257812,N2,447553408.0,11.21667,281.3884,447553.40625,8.238
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,59548.570312,N1,451916512.0,20.105261,279.454,451916.5,6.304


**3. For each (busNumber, busRoute) pair, determine the number of trips**

In [ ]:
trips.groupby(['busNumber', 'busRoute']).size()

busNumber  busRoute
183        31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
dtype: int64

**4. For each trip, compute the ratio between the energy consumption and the average number of passengers**

In [ ]:
trips['energyConsumptionKJ'] / trips['itcs_numberOfPassengers_mean']

0       86405.007812
1       12147.474609
2         73427.9375
3       167332.78125
4       26131.894531
            ...     
1404    10702.152344
1405    53384.613281
1406     47380.46875
1407    22477.525391
1408    33457.695312
Length: 1409, dtype: Float32

**5. For each station (itcs_stopName), determine the average number of passengers.**

In [ ]:
details.groupby(['itcs_stopName'])[['itcs_numberOfPassengers']].mean()

,itcs_numberOfPassengers
itcs_stopName,
"Zürich, Albisrank",16.196598
"Zürich, Albisriederplatz",25.453398
"Zürich, Altes Krematorium",21.980349
"Zürich, Bahnhof Affoltern",2.850746
"Zürich, Bahnhof Altstetten",7.816580
...,...
"Zürich, Zentrum Witikon",17.255102
"Zürich, Zweiackerstrasse",7.502203
"Zürich, Zwielplatz",2.049383


**6. For each station, determine the buses that have stopped there at least once.**

In [ ]:
stopped = details.groupby(['itcs_stopName', 'name_short']).size()
stopped[stopped>= 1]

itcs_stopName                   name_short
Zürich, Albisrank               B183           6324
                                B208           6199
Zürich, Albisriederplatz        B183          11218
                                B208           7633
Zürich, Altes Krematorium       B183          10022
                                              ...  
Zürich, Zwielplatz              B208              4
Zürich, Zwinglihaus             B183           2791
                                B208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  B183            279
                                B208            256
Length: 297, dtype: int64

**7. For each station, determine the buses that have stopped there at least ten times.**

In [ ]:
stopped[stopped >= 10]

itcs_stopName                   name_short
Zürich, Albisrank               B183           6324
                                B208           6199
Zürich, Albisriederplatz        B183          11218
                                B208           7633
Zürich, Altes Krematorium       B183          10022
                                              ...  
Zürich, Zwielplatz              B183             77
Zürich, Zwinglihaus             B183           2791
                                B208           1231
Zürich,Kalkbreite/Bhf.Wiedikon  B183            279
                                B208            256
Length: 295, dtype: int64

**8. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.**

In [ ]:
overall=(trips.groupby(['busRoute', 'busNumber'])[['drivenDistance', 'energyConsumptionKJ']].sum().apply(lambda x: x.energyConsumptionKJ / x.drivenDistance, axis=1))
overall

busRoute  busNumber
31        183          6.086892
          208          5.290863
32        183          6.174585
          208          5.491247
33        183          5.970747
          208          5.639244
46        183          5.631036
          208          5.583111
72        183          5.899030
          208          5.410319
83        183          5.844977
          208          5.819639
N1        183          5.980920
          208          5.640062
N2        183          5.701850
          208          5.405153
N4        183          6.154987
          208          6.067190
dtype: float32

**9. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.**

In [ ]:
maxmin = (overall.groupby('busRoute').agg(bus_max_ratio=lambda x: x.idxmax()[1],
                                          bus_min_ratio=lambda x: x.idxmin()[1],
                                          difference=lambda x: x.max() - x.min()))
maxmin

,bus_max_ratio,bus_min_ratio,difference
busRoute,,,
31,183,208,0.796029
32,183,208,0.683338
33,183,208,0.331503
46,183,208,0.047925
72,183,208,0.488711
83,183,208,0.025339
N1,183,208,0.340858
N2,183,208,0.296697
N4,183,208,0.087797


**10. Find the bus maximizing the difference computed in the previous point.**

In [ ]:
max_difference_row = maxmin.loc[maxmin['difference'].idxmax()]
max_difference_row

bus_max_ratio    183.000000
bus_min_ratio    208.000000
difference         0.796029
Name: 31, dtype: float32

**11. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation.**

In [ ]:
details['altitude_variation'] = details['gnss_altitude'].diff()
filtered_details = details[(details['altitude_variation'] != 0) & (details['altitude_variation'].notnull())]
filtered_details['altitude_variation']

148         0.029907
149         2.484100
150         1.375214
151        -1.203217
152        -1.366791
              ...   
48674431    0.171814
48674432    0.199890
48674433   -1.137207
48674457    0.088776
48674458    0.681000
Name: altitude_variation, Length: 40764761, dtype: float32

**12. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation.**

In [ ]:
details.groupby('name').altitude_variation.apply(lambda x: abs(x).sum())

name
B183_2019-04-30_03-18-56_2019-04-30_08-44-20      184.739075
B183_2019-04-30_13-22-07_2019-04-30_17-54-02     3896.828125
B183_2019-05-01_05-58-51_2019-05-01_22-32-30      637.407471
B183_2019-05-03_02-50-21_2019-05-03_05-53-20     4074.479004
B183_2019-05-03_15-41-57_2019-05-03_23-06-24    13325.948242
                                                    ...     
B208_2022-12-06_14-43-49_2022-12-06_18-22-52     2300.395752
B208_2022-12-07_05-13-02_2022-12-07_19-19-53     8617.676758
B208_2022-12-08_05-22-20_2022-12-08_18-39-15     8022.732422
B208_2022-12-09_23-55-12_2022-12-10_03-24-28     2748.422119
B208_2022-12-10_23-34-46_2022-12-11_03-29-05     3247.223633
Name: altitude_variation, Length: 1409, dtype: float32

**13. For each month of the year, compute the average ambient temperature**

In [ ]:
details['year']=details['time_iso'].dt.year
details['month']=details['time_iso'].dt.month
details.groupby(['year', 'month'])['temperature.celcius'].mean()

year  month
2019  4        11.592133
      5        14.262235
      6        24.686566
      7        24.222372
      8        21.506824
      9        16.888163
      10       14.333959
      11        7.942162
      12        6.471592
2020  1         6.347803
      2         6.978489
      3         8.745133
      4        16.253144
      5        16.572128
      7        24.661371
      8        24.345482
      9        21.964233
      10       11.601405
      11        8.284364
      12        5.535941
2021  1         3.556701
      2          9.24373
      3         9.181775
      4        12.957004
      5        15.510105
      6        23.096712
      7        22.862221
      8        21.107202
      9        21.014932
      10       13.272878
      11        7.463871
      12        6.967654
2022  1         5.545917
      2          7.83762
      3        11.547944
      4        13.882859
      5        20.479646
      6        23.701922
      7        26.040647
      8      

In [ ]:
trips["date"] = pd.to_datetime(trips['startTime_iso'])
trips['month'] = trips['date'].dt.month
trips.groupby("month")["celsius.mean"].mean()

month
1      5.808554
2      7.379906
3     10.169198
4      13.45825
5     16.901613
6     22.993338
7        24.227
8     22.697985
9     19.212079
10    14.564144
11     8.734443
12     6.231541
Name: celsius.mean, dtype: Float64

**14. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.**

In [ ]:
time=(details.groupby('name_short')[['status_haltBrakeIsActive', 'status_parkBrakeIsActive']].sum()
.assign(ratio=lambda x: x.status_haltBrakeIsActive / x.status_parkBrakeIsActive))
time

,status_haltBrakeIsActive,status_parkBrakeIsActive,ratio
name_short,,,
B183,7837532,3184639,2.461043
B208,5942139,1828676,3.249421


**15. For each pair of stops that are consecutive in at least a trip, compute the average speed achieved when going from the first to the second stop.**

In [ ]:
aux = {}
for i, y in details_dict.items():
    y = y.loc[
        y.itcs_stopName.first_valid_index():y.itcs_stopName.last_valid_index()]
    aux[i] = (
        y.assign(
            pair=list(zip(y.itcs_stopName.ffill(), y.itcs_stopName.bfill())))
        [['odometry_vehicleSpeed', 'pair']])

x = pd.concat(aux, ignore_index=True).groupby('pair').mean()
x[x.index.str[0] != x.index.str[1]]

,odometry_vehicleSpeed
pair,
"(Zürich, Albisrank, Zürich, Flurstrasse)",4.385559
"(Zürich, Albisrank, Zürich, Hubertus)",4.148107
"(Zürich, Albisriederplatz, Zürich, Altes Krematorium)",3.521361
"(Zürich, Albisriederplatz, Zürich, Friedhof Sihlfeld)",4.687531
"(Zürich, Albisriederplatz, Zürich, Hardplatz)",4.364703
...,...
"(Zürich, Zwinglihaus, Zürich, Goldbrunnenplatz)",3.632436
"(Zürich, Zwinglihaus, Zürich, Schmiede Wiedikon)",3.204749
"(Zürich, Zwinglihaus, Zürich,Kalkbreite/Bhf.Wiedikon)",3.752148
